In [5]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import glob
from autoreject import AutoReject, get_rejection_threshold
import eelbrain
import seaborn as sns
import pingouin as pg
import pandas as pd

# Preprocessing

## Get .vhdr files
A helper function to make the exclusions of certain participants from the analysis easier. Provide subject numbers as list of string in the exclude parameter.

In [3]:
def get_files(subjs, file_suffix, exclude):
    
    for i in range(len(exclude)):
        exclude[i] = f'./eeg_data\\Rise00{exclude[i]}.{file_suffix}'
    
    subjs = list(set(subjs) - set(exclude))
    
    return subjs

## Rename channels
Function to rename the channels in the raw files in accordance with the 10-20 standard.

In [ ]:
def rename_channels(raw, channel_names):
    channel_names_old = raw.ch_names
    channel_dict = dict(zip(channel_names_old, channel_names))
    mne.rename_channels(raw.info, mapping=channel_dict)

## Get events

### Events for subjects 4 onwards
A different function is needed to get and clean the events for subjects 4 and above because the stimuli were presented using Python and thus have different marker IDs and encoding.

In [ ]:
def get_events_main(raw):
    events_from_annot, event_dict = mne.events_from_annotations(raw)

    # Get indices of R11 events with event code 1011 or New Segment indices with event code 99999 and delete them
    useless_events = list(filter(lambda i: events_from_annot[:, 2][i] == 99999 or events_from_annot[:, 2][i] == 1011, range(len(events_from_annot[:, 2]))))
    events = np.delete(events_from_annot, useless_events, 0)

    # Get indices of consective equal events and keep only the 1st one and delete the others
    consecutive_equal_events = list(filter(lambda i: events[:, 2][i] == events[:, 2][i+1], range(len(events[:, 2])-1)))
    consecutive_equal_events = [index+1 for index in consecutive_equal_events]
    events = np.delete(events, consecutive_equal_events, 0)

    # If experiment starts with an S event with event code 2, delete it
    if events[:, 2][0] == 2:
        events = np.delete(events, 0, 0)

    # Change S event so it takes event code from R events (for e.g.- if R3 is followed by S2, we will change it to S3)
    for i in range(0, len(events), 2):
        events[i+1][2] = events[i][2]

    # Extract S events into different array
    s_events = events[1::2]

    return s_events

### Events for subjects 1, 2 and 3
A different function is needed to get and clean the events for subjects 1, 2 and 3 because the stimuli were presented using the software Presentation as compared to Python for the rest of the subjects.

In [ ]:
def get_events_before_4(raw):
    events_from_annot, event_dict = mne.events_from_annotations(raw)

    # Get indices of R11 events with event code 1011 or New Segment indices with event code 99999 and delete them
    useless_events = list(filter(lambda i: events_from_annot[:, 2][i] == 99999 or events_from_annot[:, 2][i] == 1011 or events_from_annot[:, 2][i] == 2 or events_from_annot[:, 2][i] == 6, range(len(events_from_annot[:, 2]))))
    events = np.delete(events_from_annot, useless_events, 0)

    for i in range(len(events[:, 2])):
        if events[:, 2][i] == 1004 or events[:, 2][i] == 1008:
            events[:, 2][i] = 1001
        elif  events[:, 2][i] == 1012:
            events[:, 2][i] = 1002
        elif events[:, 2][i] == 1024:
            events[:, 2][i] = 1003
        elif events[:, 2][i] == 1028:
            events[:, 2][i] = 1004

    return events

## Add Stimulus Channel to Raw
We create a stimulus channel in the raw data to add the events that were extracted and cleaned above to it.

In [ ]:
def create_stim_channel(raw, events):
    raw.load_data()
    stim_data = np.zeros((1, len(raw.times)))

    # Add stimulus channel in 'raw' object's info class
    info = mne.create_info(['STI'], raw.info['sfreq'], ['stim'])
    stim_raw = mne.io.RawArray(stim_data, info)
    raw.add_channels([stim_raw], force_update_info=True)

    # Add events extracted from annotations to the stimulus channel
    raw.add_events(events, stim_channel='STI')

## Crop Raw Data
Remove parts of the raw data that are unnecessary for further analysis.

In [ ]:
def crop_data(raw, t_from, t_to):
    first_event_time = mne.find_events(raw)[0][0]
    last_event_time = mne.find_events(raw)[-1][0]
    print(first_event_time, ' ', last_event_time)

    part_to_remove_from_beginning = (first_event_time - abs(t_from*500))/1000
    part_to_remove_from_end = (last_event_time + abs(t_to*5000))/1000
    raw.crop(part_to_remove_from_beginning, part_to_remove_from_end)

## Apply Filters
Bandpass and notch filters are applied to the raw signal

In [ ]:
def apply_filter(raw):
    # Soft bandpass Butterworth filter 
    iir_params = dict(  order=2, 
                        ftype='butter', 
                        output='sos'
                    )
    iir_params = mne.filter.construct_iir_filter(   iir_params, 
                                                    f_pass=[0.1, 30], 
                                                    f_stop=None, 
                                                    sfreq=1000, 
                                                    btype='bandpass', 
                                                    return_copy=False
                                                )
    raw.filter(0.1, 30, method='iir', iir_params=iir_params)

    # Notch filter
    raw.notch_filter(   freqs=np.arange(50, 251, 50), 
                        method='fir', 
                        fir_design='firwin2'
                    )

## Autoreject
Repair bad channels and reject bad trials using the autoreject library (https://autoreject.github.io/stable/index.html)

In [ ]:
def run_autoreject(epochs):
    ar = AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, n_jobs=1, verbose=True)
    ar.fit(epochs) 
    epochs_ar, reject_log = ar.transform(epochs, return_log=True)

    return epochs_ar, reject_log

## ICA
Fit and run ICA on epochs. We reject ICs that match the EOG pattern by matching with Fp1 as proxy for an EOG channel (since we didn't have EOG electrodes in the experiment).

In [ ]:
def run_ica(epochs, reject_log, eog_proxy):
    ica = mne.preprocessing.ICA(random_state=99)
    ica.fit(epochs[~reject_log.bad_epochs])

    # Find which ICs match the EOG pattern
    eog_indices, eog_scores = ica.find_bads_eog(epochs[~reject_log.bad_epochs], ch_name=eog_proxy)
    print(f'**************** Automatically found EOG artifact ICA components: {eog_indices} ****************')

    # # Find which ICs match the EMG pattern
    # muscle_idx_auto, scores = ica.find_bads_muscle(epochs[~reject_log.bad_epochs])
    # print(f'**************** Automatically found muscle artifact ICA components: {muscle_idx_auto} ****************')

    ica.exclude = eog_indices

    ica.plot_overlay(epochs.average(), exclude=ica.exclude)
    ica.apply(epochs, exclude=ica.exclude)

## Create Evokeds from Epochs
Evokeds are MNE objects that contain data averaged over different conditions.

In [ ]:
def create_evokeds(epochs, subj):
    # Create Evoked object from epochs (an Evoked object contains the average data over all epochs)
    evoked_standard = epochs['1001'].average()
    evoked_neutral = epochs['1002'].average()
    evoked_rise = epochs['1003'].average()
    evoked_fall = epochs['1004'].average()

    mne.write_evokeds('./analysis/'+subj[17:19]+'-ave.fif', [evoked_standard, evoked_neutral, evoked_rise, evoked_fall], overwrite=True)

    evokeds = dict(standard=evoked_standard, neutral=evoked_neutral, rise=evoked_rise, fall=evoked_fall)

    return evokeds

## Plot ERPs
Function to plot ERPs for neutral, rise and fall conditions in the desired channels, in this case, Fz, Pz, Oz, AFz, POz, CPz, FCz, Cz.

In [ ]:
def plot_erps(evokeds, subj, channels):
    # Create PDF file in which to save all plots
    with matplotlib.backends.backend_pdf.PdfPages('./analysis/'+subj[17:19]+'-plots.pdf') as pdf:
    
        for channel in channels:
            fig = mne.viz.plot_compare_evokeds(evokeds, picks=channel, combine=None, time_unit='ms', ylim=dict(eeg=[-10, 10]), invert_y=True,
                                            colors=dict(standard='black', neutral='red', rise='blue', fall='green'), 
                                            styles={'standard': {'linewidth': 1}, 'neutral': {'linewidth': 1}, 'rise': {'linewidth': 1}, 'fall': {'linewidth': 1}})
            # Save plot to PDF
            pdf.savefig(fig[0])
            plt.close()

## Variables
Declare the variables that are used often in the pipeline.

In [ ]:
eeg_files = glob.glob('./eeg_data/Rise*.vhdr')
subjs_all = get_files(eeg_files, '.vhdr', exclude=[])
# subjs_excl = get_files(eeg_files, exclude=['2', '3', '22', '25', '33', '34', '36'])

channel_names = [
                    'Fp1','Fz','F3','F7','FT9','FC5','FC1','C3','T7','TP9','CP5','CP1','Pz','P3','P7','O1','Oz','O2','P4','P8','TP10','CP6',
                    'CP2','C4','T8','FT10','FC6','FC2','F4','F8','Fp2', 'AF7','AF3','AFz','F1','F5','FT7','FC3','C1','C5','TP7','CP3','P1','P5',
                    'PO7','PO3','POz','PO4','PO8','P6','P2','CPz','CP4','TP8','C6','C2','FC4','FT8','F6','AF8','AF4','F2','FCz', 'Cz'
                ]

montage = mne.channels.make_standard_montage('easycap-M1')

epoch_limits = [-0.1, 0.6]

baseline = (-0.1, 0)

channels_to_vis = ['Fz', 'Pz', 'Oz', 'AFz', 'POz', 'CPz', 'FCz', 'Cz']

output_dir = './analysis/'

## Call Functions
Iterate over all subjects and call the functions in the pipeline for each of them.

In [ ]:
for subj in subjs_all:
    raw = mne.io.read_raw_brainvision(subj, verbose=False)

    rename_channels(raw, channel_names)
    
    raw.set_montage(montage)
    raw.plot_sensors(show_names=True)

    if int(subj[17:19]) >= 4:
        events = get_events_main(raw)
    else:
        events = get_events_before_4(raw)
        
    create_stim_channel(raw, events)

    crop_data(raw, epoch_limits[0], epoch_limits[1])

    apply_filter(raw)

    mne.add_reference_channels(raw, 'Cz', copy=False)
    mne.set_eeg_reference(raw, ref_channels='average', projection=True)
    raw.apply_proj()

    raw.set_montage(montage)

    epochs = mne.Epochs(raw, events, tmin=epoch_limits[0], tmax=epoch_limits[1], preload=True, baseline=None)

    epochs_ar, reject_log = run_autoreject(epochs)
    run_ica(epochs, reject_log, 'Fp1')

    epochs.apply_baseline(baseline=baseline)
    
    epochs_ar = run_autoreject(epochs)

    epochs_ar.save('./analysis/'+subj[17:19]+'-epo.fif', overwrite=True)

    evokeds = create_evokeds(epochs_ar, subj)
    
    plot_erps(evokeds, subj, channels_to_vis)

# Grand Average

## Combine Evokeds
Combine the evokeds object for each condition for each participant to get grand average of each condition over all subjects.

In [ ]:
def get_grand_average(subjs, outfile, exclude=[]):
    standard_subjs = []
    neutral_subjs = []
    rise_subjs = []
    fall_subjs = []

    subjs = get_files(subjs, '-ave.fif', exclude=exclude)
    
    # Separate evoked objects of all subjects into different conditions
    for subj in subjs:
        standard_subjs.append(mne.read_evokeds(subj, condition='1001'))
        neutral_subjs.append(mne.read_evokeds(subj, condition='1002'))
        rise_subjs.append(mne.read_evokeds(subj, condition='1003'))
        fall_subjs.append(mne.read_evokeds(subj, condition='1004'))

    # Combine evoked objects in each condition
    standard_combined = mne.combine_evoked(standard_subjs, weights='nave')
    neutral_combined = mne.combine_evoked(neutral_subjs, weights='nave')
    rise_combined = mne.combine_evoked(rise_subjs, weights='nave')
    fall_combined = mne.combine_evoked(fall_subjs, weights='nave')

    # Save combined evoked objects of all conditions into file
    mne.write_evokeds(f'./analysis/{outfile}', [standard_combined, neutral_combined, rise_combined, fall_combined], overwrite=True)

    evokeds = dict(standard=standard_combined, neutral=neutral_combined, rise=rise_combined, fall=fall_combined)

    return evokeds

## Plot Grand Average
Plot the timecourse for the averaged signal for each condition.

In [ ]:
def plot_grand_average(evokeds, outfile, channels=[]):
    # Create PDF file in which to save all plots
    with matplotlib.backends.backend_pdf.PdfPages(f'./analysis/{outfile}') as pdf:
    
        for channel in channels:
            fig = mne.viz.plot_compare_evokeds(evokeds, picks=channel, combine=None, time_unit='ms', ylim=dict(eeg=[-5, 5]), invert_y=True,
                                            colors=dict(standard='black', neutral='red', rise='blue', fall='green'), 
                                            styles={'standard': {'linewidth': 1}, 'neutral': {'linewidth': 1}, 'rise': {'linewidth': 1}, 'fall': {'linewidth': 1}})
            # Save plot to PDF
            pdf.savefig(fig[0])
            plt.close()

In [ ]:
# Get list of evoked objects of all subjects
all_subj_evoked = glob.glob('./analysis/Rise*-ave.fif')

# Get grand average of all subjects
evokeds = get_grand_average(all_subj_evoked, 'grand-ave.fif', exclude=[])
# Get grand average with subjects 22, 25, 33, 34 and 36 excluded
# evokeds_excluded = get_grand_average(all_subj_evoked, 'excl-grand-ave.fif', exclude=['2', '3', '22', '25', '33', '34', '36'])

# Plot channels Fz, Pz, Oz, AFz, POz, CPz, FCz, Cz for all subjects
plot_grand_average(evokeds, 'grand-ave-plots.pdf', channels=channels_to_vis)
# Plot channels Fz, Pz, Oz, AFz, POz, CPz, FCz, Cz for all subjects excluding 22, 25, 33, 34 and 36
# plot_grand_average(evokeds_excluded, 'excl-grand-ave.fif', channels=['Fz', 'Pz', 'Oz', 'AFz', 'POz', 'CPz', 'FCz', 'Cz'])

# Statistics

## Create Dataset
From evokeds data of each subject, construct a pandas dataframe over which to calculate statistics.

In [ ]:
def create_df(subjs, events_id, ch, time_win, peak_func):

    data_Pz = []

    for subj in subjs:
        for event_name in events_id:
            evk = mne.read_evokeds(subj, condition=event_name, verbose=False)

            evk.pick(ch)

            data = evk.data[0][time_win[0]:time_win[1]]

            if peak_func == 'auc':
                data_Pz.append(np.sum(np.abs(data)))
            elif peak_func == 'peak':
                data[data < 0] = 0
                data_Pz.append(np.amax(data))

    subjs_for_df = sorted([int(name[17: 19]) for name in subjs])


    df = pd.DataFrame({
            'Subject': np.repeat(subjs_for_df, len(events_id)),
            'Condition': np.tile(['Neutral', 'Rise', 'Fall'], len(subjs_for_df)),
            'uV': data_Pz
        })

    return df

In [ ]:
evokeds_files = glob.glob('./analysis/Rise*-ave.fif')

subjs_all = get_files(evokeds_files, exclude=[])
# subjs_excl = get_files(evokeds_files, exclude=['2', '3', '22', '25', '33', '34', '36'])

df = create_df  (
                    subjs_all, 
                    ['1002', '1003', '1004'], 
                    'Pz', 
                    [220, 350], 
                    'auc'
                )

## Compare Condition Averages
Using the dataframe, plot graphs comparing each condition in terms of their peak amplitude (peak) or area under curve (auc). 

In [ ]:
ax = sns.lineplot(x='Condition', y = 'uV', units= "Subject", estimator=None, data = df, alpha=0.3, hue="Subject", legend=False)
sns.stripplot(x="Condition", y="uV", hue="Condition", data=df, jitter=False, alpha =0.3)
ax = sns.pointplot(x='Condition', y = 'uV', data = df)
ax.set_title('CPz AUC between Conditions w Subj. Excl.')
sns.despine()

## Repeated-measures ANOVA

In [ ]:
# Repeated measure ANOVA
pg.rm_anova(dv='uV', 
               within='Condition', 
               subject="Subject",
               data=df,
               detailed=True)

## Paired t-tests (Rise-Fall and Rise-Neutral)

In [ ]:
# Paired t-test between Rise & Neutral
pg.ttest(x=df[df.Condition=='Rise'].uV, y=df[df.Condition=='Neutral'].uV, paired=True)

# Paired t-test between Rise & Fall
pg.ttest(x=df[df.Condition=='Rise'].uV, y=df[df.Condition=='Fall'].uV, paired=True)

# Spatio-temporal Cluster Permutation

## Disable Multiprocessing (only for Windows)
There is an issue with multiprocessing in Windows which causes problems with the permutation.

In [3]:
eelbrain.configure(n_workers=False)

In [ ]:
def get_files(subjs, exclude):
    
    for i in range(len(exclude)):
        exclude[i] = f'./analysis\\Rise00{exclude[i]}-epo.fif'
    
    subjs = list(set(subjs) - set(exclude))
    
    return subjs

epoch_files = glob.glob('./analysis/Rise*-epo.fif')

subjs_all = get_files(epoch_files, '-epo.fif', exclude=[])
# subjs_excl = get_files(epoch_files, exclude=['2', '3', '22', '25', '33', '34', '36'])

In [ ]:
conds_to_compare = ['Rise', 'Fall']

## Create Eelbrain Dataset
Create an Eelbrain dataset object that contains epoch data for each subject in each condition.

In [ ]:
ds = eelbrain.Dataset()
event_dict = {
                # '1002': 'Neutral', 
                '1003': 'Rise',
                '1004': 'Fall'
            }

rows = []
for subj in subjs_all:
    for key, val in event_dict.items():
        subject = int(subj[17: 19])
        condition = val
        eeg = eelbrain.load.fiff.epochs_ndvar(mne.read_epochs(subj, verbose=False)[key], data='eeg')
        rows.append([subject, condition, eeg.mean('case')])

ds = eelbrain.Dataset.from_caselist(['subject', 'condition', 'eeg'], rows)
ds['subject'].random = True
print(ds.summary())

In [ ]:
p = eelbrain.plot.SensorMap(ds['eeg'], connectivity=True)

## Conduct Spatio-Temporal Cluster Permutation Test
The minimum temporal extent of the clusters found is set to 800ms to avoid getting very small clusters and clusters consisting of only 1 or 2 sensors.

In [ ]:
res = eelbrain.testnd.TTestRelated(
    'eeg', 'condition', conds_to_compare[0], conds_to_compare[1], match='subject', ds=ds,
    pmin=0.05,  # Use uncorrected p = 0.05 as threshold for forming clusters
    tstart=0.100,  # Find clusters in the time window from 100 ...
    tstop=0.600,  # ... to 600 ms
    mintime=0.1,
    # minsource=2
)

In [ ]:
p = eelbrain.plot.TopoButterfly(res, clip='circle')
p.plot_colorbar()
p.set_time(0.470)

## Find Clusters
Clusters above a thershold of 0.05 are selected and returned.

In [ ]:
clusters = res.find_clusters(0.05)
print(clusters)

In [ ]:
a = clusters['id'][:]

cluster_ndvars = []

for i in range(len(a)):
    cluster = res.cluster(a[i])
    cluster_ndvars.append(cluster)
    p = eelbrain.plot.TopoArray(cluster, interpolation='nearest')
    p.set_topo_ts(0.2, 0.3, 0.4)

## Plot Cluster Timecourse
Returns plots for the timecourse of the sensors found in each cluster.

In [ ]:
masks = []

for i in range(len(cluster_ndvars)):

    # sensor x sample plot with topogrpahy
    mask = cluster_ndvars[i] != 0
    masks.append(mask)
    p = eelbrain.plot.TopoArray(mask, cmap='Wistia')
    p.set_topo_ts(.280, 0.380, 0.470)

    # barplot for magnitude of each condition within a cluster
    ds['cluster_mean'] = ds['eeg'].mean(mask)
    p = eelbrain.plot.Barplot('cluster_mean', 'condition', match='subject', ds=ds, test=False, title=f'Cluster ID: {clusters[i, "id"]}, from {clusters[i, "tstart"]} to {clusters[i, "tstop"]}')

    # topography map for spatial extent of each cluster
    roi = mask.any('time')
    p = eelbrain.plot.Topomap(roi, cmap='Wistia')

    
    ds['cluster_timecourse'] = ds['eeg'].mean(roi)
    p = eelbrain.plot.UTSStat('cluster_timecourse', 'condition', match='subject', ds=ds, frame='t', title='Compare conditions (all channels within cluster)')
    # mark the duration of the spatio-temporal cluster
    p.set_clusters(clusters[[i]])

    # mark significant sensors in topographic map of difference between conditions
    time_window = (clusters[0, 'tstart'], clusters[0, 'tstop'])
    c1_topo = res.c1_mean.mean(time=time_window)
    c0_topo = res.c0_mean.mean(time=time_window)
    diff_topo = res.difference.mean(time=time_window)
    p = eelbrain.plot.Topomap([c1_topo, c0_topo, diff_topo], axtitle=[conds_to_compare[0], conds_to_compare[1], f'{conds_to_compare[0]}-{conds_to_compare[1]}'], ncol=3)
    p.mark_sensors(roi, -1)

## Temporal cluster-based test
We restrict the spatial dimension to only 2 sensors (Pz and CPz) and conduct a temporal cluster test to find significant clusters in these channels only.

In [ ]:
ds['eeg_pz'] = ds['eeg'].sub(sensor='Pz')

res_timecoure = eelbrain.testnd.TTestRelated(
    'eeg_pz', 'condition', conds_to_compare[0], conds_to_compare[1], match='subject', ds=ds,
    pmin=0.05,  # Use uncorrected p = 0.05 as threshold for forming clusters
    tstart=0.100,  # Find clusters in the time window from 100 ...
    tstop=0.600,  # ... to 600 ms
    mintime=0.1
)
clusters_pz = res_timecoure.find_clusters(0.05)
print(clusters_pz)

p = eelbrain.plot.UTSStat('eeg_pz', 'condition', match='subject', ds=ds, frame='t', title='Compare conditions (Pz)')
p.set_clusters(clusters_pz, y=0.25e-6)

In [ ]:
ds['eeg_cpz'] = ds['eeg'].sub(sensor='CPz')

res_timecoure = eelbrain.testnd.TTestRelated(
    'eeg_cpz', 'condition', conds_to_compare[0], conds_to_compare[1], match='subject', ds=ds,
    pmin=0.05,  # Use uncorrected p = 0.05 as threshold for forming clusters
    tstart=0.100,  # Find clusters in the time window from 100 ...
    tstop=0.600,  # ... to 600 ms
    mintime=0.1
)
clusters_cpz = res_timecoure.find_clusters(0.05)
print(clusters_cpz)

p = eelbrain.plot.UTSStat('eeg_cpz', 'condition', match='subject', ds=ds, frame='t', title='Compare conditions (CPz)')
p.set_clusters(clusters_cpz, y=0.25e-6)